In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
all_matches = []

In [3]:
standings_url = "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats"

In [4]:
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

In [5]:
from io import StringIO
import time
from functools import reduce

In [6]:

for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    team_data = requests.get(team_url)
    try:
      matches = pd.read_html(StringIO(str(team_data.text)), match="Scores & Fixtures")[0]
      matches = matches[matches["Comp"] == "Premier League"]
    except ValueError:
        continue
    soup = BeautifulSoup(team_data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    sh_link = links[0].split('/')[:-3] + ['c9', 'shooting', f'{team_name}-Match-Logs-Premier-League']
    shooting_data = requests.get(f"https://fbref.com{'/'.join(sh_link)}")
    pas_link = links[0].split('/')[:-3] + ['c9', 'passing', f'{team_name}-Match-Logs-Premier-League']
    passing_data = requests.get(f"https://fbref.com{'/'.join(pas_link)}")
    def_link = links[0].split('/')[:-3] + ['c9', 'defense', f'{team_name}-Match-Logs-Premier-League']
    defense_data = requests.get(f"https://fbref.com{'/'.join(def_link)}")
    poss_link = links[0].split('/')[:-3] + ['c9', 'possession', f'{team_name}-Match-Logs-Premier-League']
    possession_data = requests.get(f"https://fbref.com{'/'.join(poss_link)}")
    try:
        shooting = pd.read_html(StringIO(str(shooting_data.text)), match="Shooting")[0]
    except ValueError:
        continue
    try:
        passing = pd.read_html(StringIO(str(passing_data.text)), match="Passing")[0]
    except ValueError:
        continue
    try:
        defense = pd.read_html(StringIO(str(defense_data.text)), match="Defensive Actions")[0]
    except ValueError:
        continue
    try:
        possession = pd.read_html(StringIO(str(possession_data.text)), match="Possession")[0]
    except ValueError:
        continue
    shooting.columns = shooting.columns.droplevel()
    passing.columns = passing.columns.droplevel()
    defense.columns = defense.columns.droplevel()
    possession.columns = possession.columns.droplevel()
    try:
        shooting_data = shooting[["Date", "Sh", "SoT", "Dist", "SoT%", "G/Sh", "G/SoT", "npxG", "npxG/Sh", "G-xG", "np:G-xG"]]
        passing_data = passing[["Date", "Cmp", "Att", "Cmp%", "TotDist", "PrgDist", "Ast", "xAG", "xA", "KP"]]
        defense_data = defense[["Date", "Tkl", "TklW","Def 3rd","Mid 3rd", "Att 3rd", "Blocks", "Sh", "Pass", "Int", "Tkl+Int", "Clr", "Err"]]
        possession_data = possession[["Date", "Touches", "Def Pen", "Def 3rd", "Mid 3rd", "Att 3rd", "Att Pen", "Live"]]
        data_frames = [matches, shooting_data, passing_data, defense_data, possession_data]
        
        teams_data = reduce(lambda  left,right: pd.merge(left,right,on='Date'), data_frames)
        # matches1 = matches.merge(shooting, on="Date")
        # matches2 = matches1.merge(passing, on="Date")
        # matches3 = matches2.merge(defense, on="Date")
        # teams_data = matches3.merge(possession, on="Date")
    except ValueError:
        continue

    teams_data["Season"] = "2021-2022"
    teams_data["Team"] = team_name
    all_matches.append(teams_data)
    time.sleep(1)

In [7]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,err,touches,def pen,def 3rd_y,mid 3rd_y,att 3rd_y,att pen,live,season,team
0,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,0,690,37,126,369,201,35,690,2021-2022,Manchester City
1,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,0,849,21,70,459,323,46,849,2021-2022,Manchester City
2,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,0,879,34,145,387,350,45,879,2021-2022,Manchester City
3,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,0,712,38,144,346,228,44,712,2021-2022,Manchester City
4,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,...,0,729,61,219,343,172,27,729,2021-2022,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2022-04-30,15:00,Premier League,Matchweek 35,Sat,Away,L,0,2,Aston Villa,...,0,636,75,257,266,120,17,636,2021-2022,Norwich City
34,2022-05-08,14:00,Premier League,Matchweek 36,Sun,Home,L,0,4,West Ham,...,1,498,58,172,214,119,16,498,2021-2022,Norwich City
35,2022-05-11,19:45,Premier League,Matchweek 21,Wed,Away,L,0,3,Leicester City,...,0,527,96,257,184,94,18,527,2021-2022,Norwich City
36,2022-05-15,14:00,Premier League,Matchweek 37,Sun,Away,D,1,1,Wolves,...,0,470,95,248,176,57,13,470,2021-2022,Norwich City


In [8]:
match_df.to_csv("matches21-22.csv")

array(['Manchester City', 'Arsenal', 'Liverpool', 'Aston Villa',
       'Tottenham Hotspur', 'Chelsea', 'Newcastle United',
       'Manchester United', 'West Ham United', 'Crystal Palace',
       'Brighton and Hove Albion', 'Bournemouth', 'Fulham',
       'Wolverhampton Wanderers', 'Everton', 'Brentford',
       'Nottingham Forest', 'Luton Town'], dtype=object)

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'team'],
      dtype='object')